In [ ]:
from os import environ, makedirs

input_folder = environ.get('CROSSCOMPUTE_INPUT_FOLDER', 'tests/standard/input')
output_folder = environ.get('CROSSCOMPUTE_OUTPUT_FOLDER', 'tests/standard/output')
try:
    makedirs(output_folder)
except IOError:
    pass

In [ ]:
import json
from os.path import join

settings_path = join(input_folder, 'settings.json')
d = json.load(open(settings_path, 'rt'))
d

In [ ]:
raw_place_name = d['name']
place_name = d['name'].replace('_', ' ').replace('.', ' ')
place_name

In [ ]:
json.dump({
    'place_name': place_name,
}, open(join(output_folder, 'settings.json'), 'wt'))

In [ ]:
open(join(output_folder, 'description.md'), 'wt').write(f'''
Here are commercial energy load profiles for **{place_name}** based on simulated data from
[USA Department of Energy Commercial Reference Building Models](http://energy.gov/eere/buildings/commercial-reference-buildings). 
[Raw data is available here](https://openei.org/doe-opendata/dataset/commercial-and-residential-hourly-load-profiles-for-all-tmy3-locations-in-the-united-states).
'''.strip())

In [ ]:
datasets_folder = 'datasets'
commercial_folder = join(datasets_folder, 'COMMERCIAL_LOAD_DATA_E_PLUS_OUTPUT')

In [ ]:
from glob import glob
from os.path import join

table_paths = glob(join(commercial_folder, raw_place_name + '*', '*'))

In [ ]:
import sys

def add_modules_folder(folder):
    if folder not in sys.path:
        sys.path.append(folder)
        
add_modules_folder('../../modules')

In [ ]:
from routines import (
    load_table,
    parse_timestamp,
    save_date_plot,
    save_table_description,
    save_table_plot,
    split_table)

In [ ]:
import re
from os.path import basename

customer_type = 'Commercial'
pattern = re.compile(r'RefBldg(.*)New2004')

for table_path in table_paths:
    table_name = basename(table_path)
    building_category = pattern.match(table_name).group(1).replace('-', '')
    t = load_table(table_path)
    electricity_table, gas_table = split_table(t)
    energy_packs = [
        ('Electricity', electricity_table),
        ('Gas', gas_table),
    ]
    for energy_type, energy_table in energy_packs:
        save_table_description(
            output_folder, energy_table, customer_type,
            energy_type, building_category)
        save_table_plot(
            output_folder, energy_table, customer_type,
            energy_type, building_category, place_name)
        save_date_plot(
            output_folder, '2000-01-01', '2000-01-02',
            energy_table, customer_type,
            energy_type, building_category, place_name)
        save_date_plot(
            output_folder, '2000-06-01', '2000-06-02',
            energy_table, customer_type,
            energy_type, building_category, place_name)